In [1]:
import cv2
import numpy as np
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import transforms, datasets

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from torch.optim import lr_scheduler

In [2]:
net = cv2.dnn.readNet("yolo4_file/yolov4-custom_person_2000.weights", "yolo4_file/yolov4-custom_person.cfg")
classes = []
with open("yolo4_file/person.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [116]:


from scipy import io
mat_file = io.loadmat('data/PRW-v16.04.20/PRW-v16.04.20/ID_train.mat')
mat_file['ID_train']

array([[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

In [28]:
from scipy import io
mat_file_test = io.loadmat('data/PRW-v16.04.20/PRW-v16.04.20/frame_test.mat')

In [107]:
img_name = mat_file_test['img_index_test'][6000][0][0]
img_path = "data/PRW-v16.04.20/PRW-v16.04.20/frames/" + img_name + ".JPG"
img_path2 = "data/PRW-v16.04.20/PRW-v16.04.20/annotations/" + img_name + ".jpg.mat"
mat_file_ann = io.loadmat(img_path2)

img = cv2.imread(img_path)
height, width, channels = img.shape
print(len(mat_file_ann['box_new']))

5


In [108]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [109]:
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


In [110]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [112]:
person_list = []

person_box = []

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        person_box.append(boxes[i])
        label = str(classes[class_ids[i]])
        cv2.rectangle(img, (x, y), (x + w, y + h), colors[0], 2)
        print(label)
        person = img[y:y+h, x:x+w].copy()
        person_list.append(person)
        #cv2.putText(img, label, (x, y + 30), font, 3, colors[0], 3)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

person
person
person
person
person


In [39]:
query = cv2.imread("data/PRW-v16.04.20/PRW-v16.04.20/query_box/857_c3s2_118744.JPG")
person_list.insert(0,query)

In [11]:
for i, person in enumerate(person_list) :
    
    path = "candi_folder/candi" + str(i) +".jpg"
    print(path)
    cv2.imwrite(path, person)

candi_folder/candi0.jpg
candi_folder/candi1.jpg
candi_folder/candi2.jpg
candi_folder/candi3.jpg
candi_folder/candi4.jpg


In [12]:
from scipy import io
mat_file = io.loadmat('data/PRW-v16.04.20/PRW-v16.04.20/ID_train.mat')

In [13]:
mat_file

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Feb 23 23:11:59 2016',
 '__version__': '1.0',
 '__globals__': [],
 'ID_train': array([[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
          27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
          40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
          53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
          66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
          79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
          92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
         105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
         118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
         131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
         144, 145, 146, 147, 148, 1

In [14]:
import os
from glob import glob

#파일 경로 리스트
def make_file_list():
    train_img_list = []
    for image_path in glob('candi_folder/*.jpg')[:len(person_list)] :
        
        train_img_list.append(image_path)
    
    return train_img_list

In [15]:
#이미지 불러오고 라벨링하는 클래스
from torch.utils.data import Dataset
from PIL import Image


class Custom_Img_Dataset(Dataset):

    def __init__(self, file_list, transform = None):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        
        img_path = self.file_list[index] # 데이터셋에서 파일 하나를 특정
        img = Image.open(img_path)
        
        user_id = img_path.split('.jpg')[0].split('candi')[1]
        
        if self.transform == None :
            tensor_image = img
        else :
            tensor_image = self.transform(img)
            
        return tensor_image, user_id

In [16]:
candi_dataset = Custom_Img_Dataset(make_file_list())

In [17]:
from torchvision import transforms, datasets

trainTransform_training  = transforms.Compose([ transforms.Resize((32, 32)), 
                                               #transforms.Grayscale(num_output_channels=1),
                                    transforms.ToTensor()
                                    #, transforms.Normalize((0.5,), (0.5,))
                                            
                                              ])



candi_dataset.transform = trainTransform_training

In [18]:
import torch
candi_loader = torch.utils.data.DataLoader(candi_dataset , batch_size=len(candi_dataset), shuffle=False, 
                               num_workers=0, drop_last=False)

In [19]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 4, 3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2, 2),
            Reshape((-1,4*8*8)),
            nn.Linear(4*8*8, 2*8*8),

        )
        self.decoder = nn.Sequential(
            nn.Linear(2*8*8, 4*8*8),
            nn.LeakyReLU(),
            Reshape((-1,4,8,8)),
            nn.ConvTranspose2d(4, 16, 2, stride=2),
            nn.LeakyReLU(),
            nn.ConvTranspose2d(16, 3, 2, stride=2),
            nn.Sigmoid()
            
        )
    def forward(self, x):
        out = self.encoder(x)
        out = self.decoder(out)
        return out
    
    def num_flat_features(self, x):
        size = x.size()[1:]  
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
class Reshape(nn.Module):
    def __init__(self, shape):
        super(Reshape, self).__init__()
        self.shape = shape
    def forward(self, x):
        return x.view(*self.shape)

In [21]:
model = AutoEncoder().to(device)
#model.load_state_dict(torch.load('Proposed_AutoEncoder'))
model.load_state_dict(torch.load('AutoEncoder'))
model.eval()

AutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.01)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Reshape()
    (7): Linear(in_features=256, out_features=128, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Reshape()
    (3): ConvTranspose2d(4, 16, kernel_size=(2, 2), stride=(2, 2))
    (4): LeakyReLU(negative_slope=0.01)
    (5): ConvTranspose2d(16, 3, kernel_size=(2, 2), stride=(2, 2))
    (6): Sigmoid()
  )
)

In [22]:
vector_list = []

for image_batch, person_id_batch in candi_loader : 
    
    imgs_vector = model.encoder(image_batch.to(device))
    
    for img_vector, person_id in zip(imgs_vector, person_id_batch) :
        vector_list.append(img_vector.detach().cpu().numpy())

In [23]:
from scipy.spatial import distance


distance_list = []

for vector in vector_list[1:] :
    distance_list.append(distance.euclidean(vector_list[0], vector))

In [24]:
distance_list

[12.038512229919434, 14.653668403625488, 11.653820037841797, 9.835874557495117]

In [26]:
x, y, w, h = person_box[distance_list.index(min(distance_list))]
cv2.rectangle(img, (x, y), (x + w, y + h), colors[0], 2)

cv2.putText(img, "answer", (x, y + 30), font, 3, colors[0], 3)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()